players__dataset_creation code for past seasons

Select the season in season variable, and repeat the code if needed

In [46]:
season = '2122'

In [45]:
import pandas as pd

In [47]:
rcsv = pd.read_csv('fbref_data/season' + season + '/outfield_players.csv')   
outfield_players = pd.DataFrame(rcsv)

rcsv = pd.read_csv('fbref_data/season' + season + '/keepers_players.csv')   
keeper_players = pd.DataFrame(rcsv)

In [48]:
players = pd.concat(   [ outfield_players[['player', 'team']], keeper_players[['player', 'team']] ], axis = 0, ignore_index = True)

players

,player,team
0,Tammy Abraham,Roma
1,Francesco Acerbi,Lazio
2,Michel Aebischer,Bologna
3,Felix Afena-Gyan,Roma
4,Kevin Agudelo,Spezia
...,...,...
676,Ciprian Tătărușanu,Milan
677,Pietro Terracciano,Fiorentina
678,Guglielmo Vicario,Empoli
679,Jeroen Zoet,Spezia


In [49]:
import unicodedata

def normalize_name(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('utf-8')

In [50]:
players['surname'] = players['player']
players['initial'] = players['player']

for i in range(players.shape[0]):
    players['surname'][i] = players['surname'][i].split(' ')[-1]
    players['surname'][i] = normalize_name(players['surname'][i]).replace('\'', '')
    
    
    players['initial'][i] = players['player'][i][0]

In [51]:
print(players[['player', 'surname']].to_string())

                         player           surname
0                 Tammy Abraham           Abraham
1              Francesco Acerbi            Acerbi
2              Michel Aebischer         Aebischer
3              Felix Afena-Gyan        Afena-Gyan
4                 Kevin Agudelo           Agudelo
5                      Ola Aina              Aina
6                    Marley Aké               Ake
7        Jean-Daniel Akpa-Akpro        Akpa-Akpro
8                  Luis Alberto           Alberto
9                Giorgio Altare            Altare
10                  Wisdom Amey              Amey
11                 Kelvin Amian             Amian
12                 Nadiem Amiri             Amiri
13                 Ethan Ampadu            Ampadu
14               Sofyan Amrabat           Amrabat
15              Felipe Anderson          Anderson
16               Ebenezer Annan             Annan
17             Cristian Ansaldi           Ansaldi
18                Janis Antiste           Antiste


In [52]:
rcsv = pd.read_csv('config/name_fix.txt')   
name_fix = pd.DataFrame(rcsv)

for i in range(name_fix.shape[0]):
    for j in range(players.shape[0]):
        if(players['surname'][j].lower() == name_fix['FROM'][i].lower() and players['team'][j].lower() == name_fix['TEAM'][i].lower()):
            players['surname'][j] = name_fix['TO'][i]
            print(name_fix['TO'][i])

name_fix


,FROM,TO
0,stigard,Ostigard
1,Min-jae,Kim
2,Hjlund,Hojlund
3,Gytkjr,Gytkjaer
4,Carlos,Augusto


In [53]:
fc_data = pd.read_excel('fantacalcio/season' + season + '/Quotazioni_Fantacalcio.xlsx', 'Tutti', header = 1)

fc_players = fc_data [['Id', 'R', 'Nome', 'Squadra']]

fc_players = fc_players.rename(columns = {'Id' : 'id', 'R': 'r', 'Nome' : 'name', 'Squadra' : 'team'})

fc_players['surname'] = fc_players['name']
fc_players['initial'] = fc_players['name']


for i in range(fc_players.shape[0]):
    spl = normalize_name( fc_players['name'][i].replace('\'', '') ).split(' ')
    if('.' in spl[-1]):
        fc_players['surname'][i] = spl[-2]
        fc_players['initial'][i] = spl[-1][0]
    else:
        fc_players['surname'][i] = spl[-1]
        fc_players['initial'][i] = ''
    
fc_players



C:\Users\nicol\AppData\Local\Temp\ipykernel_11044\2214747983.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['surname'][i] = spl[-1]
C:\Users\nicol\AppData\Local\Temp\ipykernel_11044\2214747983.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['initial'][i] = ''
C:\Users\nicol\AppData\Local\Temp\ipykernel_11044\2214747983.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['surname'][i] = spl[-

,id,r,name,team,surname,initial
0,4312,P,Maignan,Milan,Maignan,
1,453,P,Szczesny,Juventus,Szczesny,
2,2468,P,Ospina,Napoli,Ospina,
3,250,P,Handanovic,Inter,Handanovic,
4,316,P,Berisha,Torino,Berisha,
...,...,...,...,...,...,...
545,5391,A,Kokorin,Fiorentina,Kokorin,
546,5458,A,Munteanu,Fiorentina,Munteanu,
547,5459,A,Buksa,Genoa,Buksa,
548,5505,A,Kaio Jorge,Juventus,Jorge,


In [54]:
fc_players['fb_ID'] = fc_players['id']

for i in range(fc_players.shape[0]):
    fc_players['fb_ID'][i] = -1
    
    for j in range(players.shape[0]):
        if(fc_players['team'][i].lower() in players['team'][j].lower()):
            if(fc_players['surname'][i].lower() == players['surname'][j].lower()):
                # if(fc_players['initial'][i] == '' or fc_players['initial'][i].lower() == players['initial'][j].lower()):
                fc_players['fb_ID'][i] = j
                
        

C:\Users\nicol\AppData\Local\Temp\ipykernel_11044\1456862974.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['fb_ID'][i] = -1
C:\Users\nicol\AppData\Local\Temp\ipykernel_11044\1456862974.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_players['fb_ID'][i] = j


In [55]:
for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] == -1):
        print(fc_players['name'][i])

Mirante
Ujkani
Marchetti
Cordaz
Aresti
Santurro
Fuzato
Rosati
Boer
Gasparini
Furlan
Adamonis
Bertinato
Molla
Piana
Neri
Kjaer
Maehle
Ostigard
Fares
Dalbert
Czyborra
Romagna
Ballarini
De Winter
Ruiz
Djuricic
Gudmundsson A.
Arthur
Galdames
Ciervo
Sigurdsson A.
Kingsley
Obiang
Akpa Akpro
Leo' Sena
Rojas
Pecile
Bianco
Cabral
Djuric
Keita B.
Supryaga
Jovane
Edera
Munteanu
Lazetic


In [56]:
fc_players

,id,r,name,team,surname,initial,fb_ID
0,4312,P,Maignan,Milan,Maignan,,647
1,453,P,Szczesny,Juventus,Szczesny,,675
2,2468,P,Ospina,Napoli,Ospina,,653
3,250,P,Handanovic,Inter,Handanovic,,644
4,316,P,Berisha,Torino,Berisha,,637
...,...,...,...,...,...,...,...
545,5391,A,Kokorin,Fiorentina,Kokorin,,283
546,5458,A,Munteanu,Fiorentina,Munteanu,,-1
547,5459,A,Buksa,Genoa,Buksa,,83
548,5505,A,Kaio Jorge,Juventus,Jorge,,266


In [57]:
#Data for Outfield players
columns_to_copy = outfield_players.columns[4:]

fc_players[columns_to_copy] = 0

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] != -1 and fc_players['r'][i] != 'P'):
         for j in range(columns_to_copy.shape[0]):
            fc_players[columns_to_copy[j]][i] = outfield_players[columns_to_copy[j]][fc_players['fb_ID'][i]]
            

C:\Users\nicol\AppData\Local\Temp\ipykernel_11044\2261782218.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy] = 0
C:\Users\nicol\AppData\Local\Temp\ipykernel_11044\2261782218.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy] = 0
C:\Users\nicol\AppData\Local\Temp\ipykernel_11044\2261782218.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns a

In [58]:
keeper_players.columns[4:]

Index(['age', 'birth_year', 'gk_games', 'gk_games_starts', 'gk_minutes',
       'gk_goals_against', 'gk_goals_against_per90',
       'gk_shots_on_target_against', 'gk_saves', 'gk_save_pct', 'gk_wins',
       'gk_ties', 'gk_losses', 'gk_clean_sheets', 'gk_clean_sheets_pct',
       'gk_pens_att', 'gk_pens_allowed', 'gk_pens_saved', 'gk_pens_missed',
       'minutes_90s', 'gk_free_kick_goals_against',
       'gk_corner_kick_goals_against', 'gk_own_goals_against', 'gk_psxg',
       'gk_psnpxg_per_shot_on_target_against', 'gk_psxg_net',
       'gk_psxg_net_per90', 'gk_passes_completed_launched',
       'gk_passes_launched', 'gk_passes_pct_launched', 'gk_passes',
       'gk_passes_throws', 'gk_pct_passes_launched', 'gk_passes_length_avg',
       'gk_goal_kicks', 'gk_pct_goal_kicks_launched',
       'gk_goal_kick_length_avg', 'gk_crosses', 'gk_crosses_stopped',
       'gk_crosses_stopped_pct', 'gk_def_actions_outside_pen_area',
       'gk_def_actions_outside_pen_area_per90', 'gk_avg_distance_

In [59]:
#Data for Keepers
columns_to_copy = keeper_players.columns[4:]

fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year

delta_k = outfield_players.shape[0]

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] != -1 and fc_players['r'][i] == 'P'):
        if(fc_players['fb_ID'][i] - delta_k >= 0):
            for j in range(columns_to_copy.shape[0]):          
                fc_players[columns_to_copy[j]][i] = keeper_players[columns_to_copy[j]][fc_players['fb_ID'][i] - delta_k]

C:\Users\nicol\AppData\Local\Temp\ipykernel_11044\3175819694.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year
C:\Users\nicol\AppData\Local\Temp\ipykernel_11044\3175819694.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year
C:\Users\nicol\AppData\Local\Temp\ipykernel_11044\3175819694.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is us

In [60]:
keeper_players

,player,nationality,position,team,age,birth_year,gk_games,gk_games_starts,gk_minutes,gk_goals_against,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,Emil Audero,it ITA,GK,Sampdoria,24,1997,29.0,29.0,2560.0,48.0,...,36.3,212.0,67.9,47.6,418.0,22.0,5.3,25.0,0.88,13.5
1,Nicola Bagnolini,it ITA,GK,Bologna,17,2004,1.0,0.0,3.0,0.0,...,61.0,1.0,100.0,60.0,3.0,0.0,0.0,0.0,0.00,0.0
2,Francesco Bardi,it ITA,GK,Bologna,29,1992,2.0,2.0,177.0,2.0,...,33.0,13.0,38.5,32.1,24.0,2.0,8.3,0.0,0.00,5.5
3,Vid Belec,si SVN,GK,Salernitana,31,1990,23.0,21.0,1938.0,49.0,...,39.5,182.0,73.1,49.9,329.0,12.0,3.6,14.0,0.65,12.7
4,Alessandro Berardi,it ITA,GK,Hellas Verona,30,1991,1.0,1.0,90.0,3.0,...,39.7,7.0,100.0,58.7,12.0,0.0,0.0,0.0,0.00,0.0
5,Etrit Berisha,al ALB,GK,Torino,32,1989,10.0,10.0,900.0,8.0,...,39.4,78.0,87.2,58.6,148.0,7.0,4.7,10.0,1.00,14.6
6,Andrea Consigli,it ITA,GK,Sassuolo,34,1987,37.0,37.0,3322.0,63.0,...,30.9,246.0,31.7,28.8,491.0,21.0,4.3,30.0,0.81,13.4
7,Alessio Cragno,it ITA,GK,Cagliari,27,1994,35.0,35.0,3150.0,63.0,...,39.4,316.0,73.4,48.2,484.0,22.0,4.5,33.0,0.94,15.0
8,Bartłomiej Drągowski,pl POL,GK,Fiorentina,23,1997,7.0,7.0,556.0,8.0,...,30.8,39.0,43.6,40.2,61.0,2.0,3.3,20.0,3.24,22.3
9,Wladimiro Falcone,it ITA,GK,Sampdoria,26,1995,10.0,9.0,855.0,14.0,...,33.2,74.0,70.3,45.0,164.0,7.0,4.3,2.0,0.21,10.2


In [61]:
fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,4312,P,Maignan,Milan,Maignan,,647,26,1995,0,...,33.0,149,38.9,36.5,347,25,7.2,45,1.41,16.8
1,453,P,Szczesny,Juventus,Szczesny,,675,31,1990,0,...,30.8,172,29.7,31.5,481,24,5.0,25,0.76,13.5
2,2468,P,Ospina,Napoli,Ospina,,653,32,1988,0,...,28.6,135,14.1,22.5,353,18,5.1,31,1.00,17.0
3,250,P,Handanovic,Inter,Handanovic,,644,37,1984,0,...,28.1,142,20.4,26.5,393,15,3.8,10,0.27,11.2
4,316,P,Berisha,Torino,Berisha,,637,32,1989,0,...,39.4,78,87.2,58.6,148,7,4.7,10,1.00,14.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,5391,A,Kokorin,Fiorentina,Kokorin,,283,30,1991,6,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
546,5458,A,Munteanu,Fiorentina,Munteanu,,-1,0,0,0,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
547,5459,A,Buksa,Genoa,Buksa,,83,18,2003,4,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
548,5505,A,Kaio Jorge,Juventus,Jorge,,266,19,2002,9,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0


In [62]:
import numpy as np

votes = pd.read_excel('mid_outputs/season' + season + '/players_votes.xlsx', index_col = 0)

mean_def = 6
std_def = 0.58

mean_def_P = 6.22
std_def_P = 0.43


min_votes = 6


perf_df = pd.DataFrame(columns = ['vote_avg', 'vote_std'])

for i in range(fc_players.shape[0]):
    v = np.array([])
    for j in range(votes.shape[0]):
        if(fc_players['name'][i] == votes['player'][j]):
            v = np.append(v, votes['vote'][j])
    
    if(fc_players['r'][i] == 'P'):
        mean_def_i = mean_def_P
        std_def_i = std_def_P
        
    if(v.shape[0] < min_votes):
         for k in range(min_votes - v.shape[0]):
            v = np.append( v, np.random.normal(mean_def_i, std_def_i) )
    
    row_df = pd.DataFrame(data = [[np.mean(v), np.std(v)]], columns = perf_df.columns)
    perf_df = pd.concat([perf_df, row_df], ignore_index = True)
    
perf_df
    

,vote_avg,vote_std
0,6.265625,0.483871
1,6.121212,0.628012
2,6.258065,0.332899
3,6.162162,0.533399
4,6.100000,0.700000
...,...,...
545,5.901518,0.269227
546,6.215233,0.275498
547,6.035319,0.591212
548,6.137790,0.468234


In [63]:
fc_players = pd.concat([fc_players, perf_df], axis = 1)

fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions,vote_avg,vote_std
0,4312,P,Maignan,Milan,Maignan,,647,26,1995,0,...,38.9,36.5,347,25,7.2,45,1.41,16.8,6.265625,0.483871
1,453,P,Szczesny,Juventus,Szczesny,,675,31,1990,0,...,29.7,31.5,481,24,5.0,25,0.76,13.5,6.121212,0.628012
2,2468,P,Ospina,Napoli,Ospina,,653,32,1988,0,...,14.1,22.5,353,18,5.1,31,1.00,17.0,6.258065,0.332899
3,250,P,Handanovic,Inter,Handanovic,,644,37,1984,0,...,20.4,26.5,393,15,3.8,10,0.27,11.2,6.162162,0.533399
4,316,P,Berisha,Torino,Berisha,,637,32,1989,0,...,87.2,58.6,148,7,4.7,10,1.00,14.6,6.100000,0.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,5391,A,Kokorin,Fiorentina,Kokorin,,283,30,1991,6,...,0.0,0.0,0,0,0.0,0,0.00,0.0,5.901518,0.269227
546,5458,A,Munteanu,Fiorentina,Munteanu,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.215233,0.275498
547,5459,A,Buksa,Genoa,Buksa,,83,18,2003,4,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.035319,0.591212
548,5505,A,Kaio Jorge,Juventus,Jorge,,266,19,2002,9,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.137790,0.468234


In [64]:
min_gk_games = 6

fc_players_newgk = fc_players.copy()

columns_to_avg = fc_players.columns[123:] # from gk_games to end

for i in range(fc_players.shape[0]):
    if(fc_players['r'][i] == 'P'):
        if(fc_players['gk_games'][i] < min_gk_games):
            for j in range(fc_players.shape[0]):
                if(fc_players['team'][i] == fc_players['team'][j] and fc_players['gk_games'][j] >= min_gk_games):
                    break
                    
            weight = 1 - (min_gk_games - fc_players['gk_games'][i]) / min_gk_games
            
            fc_players_newgk.at[i, columns_to_avg] = fc_players.loc[i][columns_to_avg] * weight + (1 - weight) * fc_players.loc[j][columns_to_avg]
            
            print(fc_players['name'][i] + ', ' + str(weight))
            
        

Sportiello, 0.8333333333333334
Bardi, 0.33333333333333337
Padelli, 0.5
Pinsoglio, 0.16666666666666663
Semper, 0.16666666666666663
Zovko, 0.16666666666666663
Radunovic, 0.5
Perin, 0.8333333333333334
Mirante, 0.0
Ujkani, 0.0
Marchetti, 0.0
Pegolo, 0.16666666666666663
Radu I., 0.16666666666666663
Cordaz, 0.0
Aresti, 0.0
Fiorillo, 0.16666666666666663
Satalino, 0.16666666666666663
Santurro, 0.0
Rossi F., 0.16666666666666663
Fuzato, 0.0
Rosati, 0.0
Berardi A., 0.16666666666666663
Gemello, 0.16666666666666663
Ravaglia, 0.16666666666666663
Pandur, 0.5
Boer, 0.0
Gasparini, 0.0
Furlan, 0.0
Adamonis, 0.0
Marfella, 0.16666666666666663
Bertinato, 0.0
Molla, 0.0
Russo, 0.0
Piana, 0.0
Neri, 0.0


In [65]:
fc_players = fc_players_newgk

fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions,vote_avg,vote_std
0,4312,P,Maignan,Milan,Maignan,,647,26,1995,0,...,38.9,36.5,347.0,25.0,7.2,45.0,1.41,16.8,6.265625,0.483871
1,453,P,Szczesny,Juventus,Szczesny,,675,31,1990,0,...,29.7,31.5,481.0,24.0,5.0,25.0,0.76,13.5,6.121212,0.628012
2,2468,P,Ospina,Napoli,Ospina,,653,32,1988,0,...,14.1,22.5,353.0,18.0,5.1,31.0,1.00,17.0,6.258065,0.332899
3,250,P,Handanovic,Inter,Handanovic,,644,37,1984,0,...,20.4,26.5,393.0,15.0,3.8,10.0,0.27,11.2,6.162162,0.533399
4,316,P,Berisha,Torino,Berisha,,637,32,1989,0,...,87.2,58.6,148.0,7.0,4.7,10.0,1.00,14.6,6.100000,0.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,5391,A,Kokorin,Fiorentina,Kokorin,,283,30,1991,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,5.901518,0.269227
546,5458,A,Munteanu,Fiorentina,Munteanu,,-1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,6.215233,0.275498
547,5459,A,Buksa,Genoa,Buksa,,83,18,2003,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,6.035319,0.591212
548,5505,A,Kaio Jorge,Juventus,Jorge,,266,19,2002,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,6.137790,0.468234


In [67]:
fc_players.to_excel('mid_outputs/season' + season + '/players_stats.xlsx')